# Halo, Landmarks and Classes

This notebook will go through three steps. Below we list the output of these steps.
* Producing Halos. These are voxels levitated away from a protein at 2.5 Å but within 5.0Å of the protein; only heavy atoms are considered in the protein. They are stored as indices of a sparse tensor.
* Producing Landmarks. These are positions indicating the presence of a certain class e.g. for being a nucleotide, part of pocket, etc. Redundance is also indicated here. See notes below.
* Typifying Halos. This process will assign classes to halos, but in the passing it will also filter away halos belonging to component weakly interacting with the protein i.e. components not touching any protein atoms within 3.0 Å.

Note that the unit size of the voxel is defined by grid points on a cubic lattice spaced at 1 Å; this is reasonable considering sqaure root of 3 (1.7Å) being the lower margin of non-covalent contact. We will explain these steps en-route below.




## Imports and File I/O
This sections define necessary imports and directories. Safe click!

In [1]:
# =================== Imports
import sys
import glob
import tqdm
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append('../')



## Generate Halo
We will use the term Halo to generalise a collection of voxels/coordinates levitated away from the protein surface (between 2.5 Å to 5.0Å) without colliding with any of its heavy atoms. Here, we store and organise the halo in a sparse format. The output `{pdbid}.halotup` is a tuple containing 4 items `halo_tuple = (halo_index, lattice_shape, lattice_minmax, halo_bound)`:
* `halo_index`. Indices of the Halo, which is a subset of indices in the lattice. Integers
* `lattice_shape`. The shape of the lattice. We only consider cubic lattice. This is the shape of a 3D array. 
* `lattice_minmax`. The min/max position of the lattice to produce the Halo. Together with the indices, this will allow us to retrieve the coordinate of each halo when needed. Floats.
* `halo_bound`. This is a tuple stating the boundary on the protein surface as `(HaloLowerBound, HaloUpperBound)`.

In addition to the tuple, the coordinate of all halos can also be generated together as floats in file `{pdbid}.haloxyz` using the flag `WritingXyz=True`

Below starts a massive production of halo. For 12000 NA-bound pdb it took 2 hours w/ 10 processor. You will also need 20 GB in storage for just halo tuples and an additional 29GB when halo coordinates are to be generated. While the storage it takes seems astounding, it will facillitate the following processes:
* Featurisations to be done on a coordinate/voxel
* Message passing in more sophisticated training scheme among nearby voxels in terms of label smoothing (and maybe in terms of the features in the future). 


In [2]:
from NucleicNet.DatasetBuilding.commandHalo import *

HaloC = Halo(HaloLowerBound = 2.5, HaloUpperBound = 5.0, LatticeSpacing = 1.0, 
             DIR_OutputHaloTupleFolder = "../Database-PDB/halo",
             DIR_InputPdbFolder = "../Database-PDB/apo", 
             DIR_OutputTypifyFolder = "../Database-PDB/typi")

# =====================================
# An example of Halo 
# =======================================
#halo_tuple = HaloC.Voxelise(PlottingHalo = True, WritingXyz = False,
#                        DIR_JustOnePdb = "../Database-PDB/apo/6uol00000000.pdb", 
#                        SavingHalo = True, UpdateExist = True)
#halo_coord = HaloC.RetrieveHaloCoords(halo_tuple)


# ===================================
# Massive production of Halo
# ====================================
HaloC.Voxelise(PlottingHalo = False, WritingXyz = True,
                        DIR_JustOnePdb = None, 
                        SavingHalo = True, UpdateExist = False)
#halo_coord = HaloC.RetrieveHaloCoords(halo_tuple)


## Generate Landmark
We use the term 'Landmarks' to generalise on a collection of coordinates corresponding to certain classes. For example, they can indicate where pockets (`F` location of concave surface fulfilling certain criteria) are established, where NA-binding sites (e.g. `A`, `U`, `C`, `G`) are established, etc. Two types of landmarks are provided in this implementation

* Fpocket landmarks. These are coordinates of the remaining alpha spheres filtered by [Fpocket](https://www.google.com/search?channel=fs&client=ubuntu&q=Fpcoket)
* Nucleotide landmarks. These are centroids of base and backbone components. 

Landmarks indicates presence of some classes. In some cases, we may want to remove internally redundant landmarks (i.e. those coming from copies of the same/almost-exact-matched nucleotide chain) and keep only a non-redundant copy (marked with a suffix `_`), so that a fair efficient validation dataset can be constructed. In the current implementation, a simple edit distance is used to recognise matching nucleotide chains in a PDB entry. We will illustrate this in next cell. 

> LandmarkC.Nucleotide(   
                        DIR_NaBoundPdbFolder = "../Database-PDB/cleansed",
                        ProteinContactDistanceThreshold = 5.0,
                        AtomicContactDistanceThreshold = 3.8,
                        EditDistanceThreshold = 0.1,
                        IndicateNonRedun = True,
                        UpdateExist = False,SavingLandmark = True, 
                        PrintingVerbose = False)

A parameter worth mentioning is `AtomicContactDistanceThreshold = 3.8`. This defines how far heavy atoms of nucleotide components can be away from any protein heavy atoms when we consider it as a valid landmark interacting with the protein. 3.8 Å is the [nominal max distance](https://link.springer.com/article/10.1007/s12539-015-0263-z#Fig1) concerning face-to-face $\pi \pi$ stacking. Setting `AtomicContactDistanceThreshold = 3.8` means when there is no protein heavy atom within 3.8 Å of the nucleotide component, then the nucleotide component will not be considered as a valid landmark. The output `.nucsite.landmark` is a biopandas format dataframe containing an extra column `ProteinAtomContactNumber` to indicate how many atoms are within `AtomicContactDistanceThreshold`; invalid landmarks are will have `ProteinAtomContactNumber == 0`. (Note rather than deleting the landmark we will still keep it; this is necessary because these "placeholders" will keep the voronoi boundary clean in later steps). We will use this parameter to remove non-interacting components.

Below starts the landmarking w/ 16 processors; it will run for ~10 hours for fpocket and around ~2 hours for nucleotides, depending on your machine specs. Trivia. Also note that it raises exception for NA-bound structures that made entirely of modified base! They will be rejected. Some examples:
* 1fyk
* 4r8i
* 1kdh



In [5]:
from NucleicNet.DatasetBuilding.commandLandmark import *

LandmarkC = Landmark(DIR_InputPdbFolder = "../Database-PDB/apo", 
                DIR_OutputLandmarkFolder = "../Database-PDB/landmark",
                n_MultiprocessingWorkers = 16)

# Fpocket Landmarks
LandmarkC.Fpocket(     mode = "default", UpdateExist= False,
                SavingLandmark = True, IndicateNonRedun = True,
                EditDistanceThreshold = 0.1,
                PrintingVerbose = False)



# Nucleotide Landmarks
# NOTE ProteinContactDistanceThreshold is a loose criterion to catch chain that makes the most contact with landmark, but not necessarily atoms in the component
#      because of its confusing naming, it may be changed in later version
# NOTE AtomicContactDistanceThreshold is the maximum inter-heavy atom distance to catch protein-atom contacting component
#      Face to face stacking for heavy atoms max at ~3.8. example: https://link.springer.com/article/10.1007/s12539-015-0263-z#Fig1
LandmarkC.Nucleotide(   
                        DIR_NaBoundPdbFolder = "../Database-PDB/cleansed",
                        ProteinContactDistanceThreshold = 5.0,
                        AtomicContactDistanceThreshold = 3.8,
                        EditDistanceThreshold = 0.1,
                        IndicateNonRedun = True,
                        UpdateExist = False,SavingLandmark = True, 
                        PrintingVerbose = False)


***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
*

! File '/tmp/tmp3dsqsla1/3c5f00000001.pdb' contains no atoms...
! File '/tmp/tmp3dsqsla1/3c5f00000001.pdb' contains no atoms...
! PDB reading failed!


***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 


mv: cannot stat '/tmp/tmp3dsqsla1/3c5f00000001_out/': No such file or directory


Fpocket generates no output for 3c5f00000001
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***

exceeding memory size for each grid element
exceeding memory size for each grid element
exceeding memory size for each grid element
exceeding memory size for each grid element


***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HU

exceeding memory size for each grid element
exceeding memory size for each grid element
exceeding memory size for each grid element


***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTIN

## Examples of Redundant Landmarks
Before we move on I would like to illustrate some typical sohpistications in creating landmarks. 

* Example of a long double strand DNA (Easy)
* Example of a long double strand RNA (Easy)
* Example of a D/RNA hybrid (Hybrid)
* Example of a pdb w/ 4 copies of polyU sequence at the interface of 2 homologous protein chains. (Internal Symmetry in Nucleotides and Proteins)

These cases are handled by the landmarking protocol. 

* Internal Symmetry descibe "copies" of proteins or nucleotide in the [unit cell of the same PDB entry](https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/biological-assemblies). This can interfere/bias the later validation steps for the non-sites. 
* * During landmarking one of the "copies" will be marked as non-redundant using the class (`nucsite_`) in *.nucsite.landmark; notice the underscore suffix. This copy is chosen as the one making most contact with proteins. Also note that many of the "copies" are not exact-matches - some positions in these asymmetric units are truncated or missing such that we need to tolerate their similarities with [edit distance](https://en.wikipedia.org/wiki/Edit_distance). Note that we should never remove a homologous unit (protein/nucleotide) when we later on featurise a site, because that will remove the interfacing features - we should simply indicate them by landmarks.

Similar problems concerning redundance exist among subsets of separate PDB entries with homologous sequences; we will handle this External Symmetry in the training stage by reweighed sampling of clans.

In [4]:
from NucleicNet.DatasetBuilding.commandLandmark import *
# Example of a DRNA hybrid
OOC_Nucleotide("../Database-PDB/cleansed/2qkb00000000.pdb", PrintingVerbose=True, SavingLandmark=False)
# Example of a long double strand DNA
OOC_Nucleotide("../Database-PDB/cleansed/4c2u00000000.pdb", PrintingVerbose=True, SavingLandmark=False)
# Example of a long double strand RNA
OOC_Nucleotide("../Database-PDB/cleansed/5amr00000000.pdb", PrintingVerbose=True, SavingLandmark=False)
# Example of a pdb w/ 4 copies of polyU sequence at the interface of 2 homologous protein chains!
OOC_Nucleotide("../Database-PDB/cleansed/5gmf00000000.pdb", PrintingVerbose=True, SavingLandmark=False)


2qkb00000000 has the following nucleotide chain groups and sequences
defaultdict(<class 'list'>, {0: ['C'], 1: ['D']})
{'C': ['DG', 'G', 'A', 'G', 'U', 'G', 'C', 'G', 'A', 'C', 'A', 'C', 'C', 'U', 'G', 'A', 'U', 'U', 'C', 'C'], 'D': ['DG', 'DG', 'DA', 'DA', 'DT', 'DC', 'DA', 'DG', 'DG', 'DT', 'DG', 'DT', 'DC', 'DG', 'DC', 'DA', 'DC', 'DT', 'DC', 'DT']}
2qkb00000000 has 119 centroids. 3 centroid per nucleotide if no missing component. Make Sense?
4c2u00000000 has the following nucleotide chain groups and sequences
defaultdict(<class 'list'>, {0: ['X'], 1: ['Y']})
{'X': ['DA', 'DC', 'DG', 'DA', 'DC', 'DC', 'DT', 'DG', 'DC', 'DG', 'DA', 'DG', 'DC', 'DA', 'DC', 'DT', 'DG', 'DC', 'DT', 'DT', 'DT', 'DT', 'DT', 'DT'], 'Y': ['DC', 'DA', 'DG', 'DT', 'DG', 'DC', 'DT', 'DC', 'DG', 'DC', 'DA', 'DG', 'DG', 'DT', 'DC', 'DG', 'DT', 'DT', 'DT', 'DT', 'DT', 'DT', 'DT']}
4c2u00000000 has 141 centroids. 3 centroid per nucleotide if no missing component. Make Sense?
5amr00000000 has the following nucleoti

## Assign Classes to Voxels

Now we have collected some landmarks, we can then try to assign classes to the voxels referencing the landmarks. We call this process "typification". Some standard usage includes 
* Assignment to the Nearest Landmark Centroid with flag `TakeCentroid = True`. This assign each voxel in the halo to take the same class as the nearest landmark. This is equivalent to a Voronoi tessellation process, but we can avoid assigning voxels to nearest landmark too far away by the flag `DistanceCutoff = 3.0`, here the 3.0 angstrom is chosen such that a sphere centered at the centroid with this radius will contain all atoms in a component of nucleotide.  

* Assignment of Non-sites. These are voxels in the halo that is not within any nucleotide or designated resname. We also considered the redundance in these voxels due to the Internal Symmetry in the PDB entry. There are usually much more non-sites than sites; there are usually much more redundant non-sites than non-redundant ones. So, a natural sparse annotation is to indicate all not-non-sites as 1 in a column (not-non-site versus its complement, which is non-site) and all non-redundant non-site as 1 in another column (non-redundant non-site vs its complement, which can be redundant non-site or not-nonsite). 

Currently, the following 13 classes including indication of places of absence `nonsite_` are provided. 
> `{'F': 0, 'fpocket_': 1, 'A': 2, 'C': 3, 'D': 4, 'G': 5, 'P': 6, 'R': 7, 'T': 8, 'U': 9, 'nucsite_': 10, 'nonsite_': 11, 'notnonsite': 12}`

The following cell will start a massive typification process, which can take more than 20 hours to complete on 16 processors with ~12000 structures. It will produce a sparse class matrix (n_halovoxels, n_classes) that correspond to the halo's voxel index tuples (n_halovoxels, 3) with columns indicating the class assigned. Non-redundancy is indicated by class with a suffix underscore. 

Additional remarks:

* D/RNA Hybrids are handled by class labels `(A,G,C,T,U,"D","R",P)` in `*.nucsite.landmark`. `"D"` and `"R"` are deoxy-ribose and ribose respectively; overlapping base `A,G,C` in D/RNA are merged. Note that some NA-binding proteins can accept both D/RNA sharing the same base disregarding the difference in ribose. Also note that this same indifference pertains in the classification of bases. A critique here is that while classification among base, phosphate, ribose and nonsite is largely multi-class-single-label, the classification of AUCG can be multi-class-multi-label, though a nice pareto is achieved in our model due to variance in data label.

* Currently the matrix contains only 1 or 0 (though I store it as a float!) but certainly when sequence-based ground truth are incorporated we can take the probability. Note that we should also make effort to take covariance in sequence in account e.g. calculating a (n_lenseq,n_lenseq,4,4) covariance tensor. We will need to think about how to realise this data format. 


In [6]:
from NucleicNet.DatasetBuilding.util import *
from NucleicNet.DatasetBuilding.commandTypify import Typify

# TODO After typification we need no landmarks, landmarks takes 1 GB storage only, can we make it on memory?

TypifyC = Typify(DIR_TypiFolder = "../Database-PDB/typi",
                DIR_HaloFolder = "../Database-PDB/halo",
                )
TypifyC.Pipeline(DIR_LandmarkFolder = "../Database-PDB/landmark",
                DIR_BoundPdbFolder = "../Database-PDB/cleansed",
                         NotNonsiteResnameList = ["DA","DT","DC","DG","DU",
                                                    "A","T","C","G","U",
                                                    # NOTE also accept other modified base and possibly ligand names if wanted
                                                    "1MG","5MC","1MA","U37","UFT",
                                                    "SUR","PYO","P5P","OMC","OMG","OMU","ONE",
                                                    "FMU","GTA","CFZ","CFL","AP7","ADN","AZM",
                                                    "A9Z","6MZ","N5M","5BU","2PR","2AD","23G","A23",
                                                    "APC","CGI","PGP","U5P",
                                                    "DI","I"], 
                         Recipe = ["landmark", "nonsite"],
                         DistanceCutoff = 3.0, # NOTE This is the distance angstrom cutoff to decide how concentrate are we w.r.t. each site centroid. 3.0 Accuracy ~50% 3CV
                         NotNonsiteDistanceCutoff = 5.0, # NOTE This is the distance within which we consier as not nonsite.
                         TakeCentroid = True, SavingTypi = True, UpdateExist = False)



The following pdbid were excluded. Check
['1a1v00000000', '1kdh00000000', '1m0600000000', '2f5500000000', '3af600000000', '3c5f00000001', '3f2100000000', '3f2200000000', '3f2300000000', '3iem00000000', '3vaf00000000', '3vaf00000001', '3vak00000000', '3vak00000001', '4r8i00000000', '4tu700000000', '4tu700000001', '4wb200000000', '4wb200000001', '4wb300000000', '4wb300000001', '6idg00000000', '6kcp00000000', '6kdi00000000', '6l9700000000', '6mdx00000000', '6mdx00000001', '6u6x00000000', '6u6x00000001', '6ycs00000000', '6ycs00000001']
There are 1626 PDB entries to process
{'F': 0, 'fpocket_': 1, 'A': 2, 'C': 3, 'D': 4, 'G': 5, 'P': 6, 'R': 7, 'T': 8, 'U': 9, 'nucsite_': 10, 'nonsite_': 11, 'notnonsite': 12}
4wsb00000000 has no halo. Why? Check. ABORTED.


## Descriptive Statistics

Below are some descriptive stats for the class labels. Stacks are made of different pdb entries. Note that class with an underscore `"_"` are non-redundant indicator. Some observations:
* Some classes (phosphate, deoxyribose) are more abundant. This may potentially lead to an imbalanced training if not handled carefully. This will be handled with pyotrch's systematic dataset/dataloader scheme to avoid obtaining a hard-to-interpret Pareto optimum.
* There are much more non-sites than any other class (and we are only counting the non-redundant non-sites!). Again, this has to be handled carefully in training. A leeway is to stride the dataset so that it fits to memory.
* Some of the PDB entries have their NucleicAcid labelled "nan" on the FTP. For example
* * '7ox7', '7ox8', '7ox9', '7oxa' has a chimeric fusion of RNA and DNA
* * '6vf0', '6vf1', '6vf2', '6vf3', '6vf4', '6vf5', '6vf6', '6vf7', '6vf8', '6vf9', '6vfa', '6vfb', '6vfc', '7d3t', '7d3v', '7d3w', '7d3x', '7l4v', '7l4x', '7l4y', '7lj3', '7lxh', '7lxj', '7om3', '7omb', '7omg', '7ox7', '7ox8', '7ox9', '7oxa', '7rfk', '7rfl', '7rfm', '7rfn', etc are recently indexed entries.


In [25]:
%config InlineBackend.figure_format = 'svg'
sns.set_context("notebook")

# Loading pdb info
Df_grand = pd.read_pickle("../Database-PDB/DerivedData/DataframeGrand.pkl")

# Classes available in the Typi
with open("../Database-PDB/typi/ClassIndex.pkl", "rb") as fn:
    tcmapping = pickle.load(fn)
tcmapping_ = {v:k for k,v in tcmapping.items()}
labellist = [tcmapping_[i] for i in sorted(tcmapping_.keys())]
print(tcmapping)

# Collection of statistics
labelstat_df_ = []
pdbid_natype = {}
for typi in tqdm.tqdm(sorted(glob.glob("../Database-PDB/typi/*.typi"))):
    pdbid = typi.split("/")[-1].split(".")[0]
    natype = str(Df_grand.loc[Df_grand['Pdbid'] == pdbid]['NucleicAcid'].tolist()[0])

    pdbid_natype[pdbid] = natype
    with open(typi, "rb") as fn:
        SparseClassMatrix = pickle.load(fn)
    classsum = np.array(SparseClassMatrix.todense().sum(axis =0)).flatten().tolist()
    tempstat = [pdbid]
    tempstat.extend(classsum)

    labelstat_df_.append(tempstat)


labelstat_df = pd.DataFrame(labelstat_df_, columns = ['Pdbid']+labellist)

labelstat_df = pd.melt(labelstat_df, id_vars=['Pdbid'], value_vars=labellist)
labelstat_df['NucleicAcid'] = labelstat_df['Pdbid'].map(pdbid_natype)
print(sorted(set(labelstat_df.loc[labelstat_df['NucleicAcid'] =='nan']['Pdbid'].tolist())))
#labelstat_df = labelstat_df.dropna(subset=['NucleicAcid']) #NOTE some pdbid were not in grand dataframe?
labelstat_df = labelstat_df.rename(columns={"variable":"Classes", "value":"Voxel Count"})
#print(labelstat_df)
# Plotting the stat
from matplotlib.colors import ListedColormap
f, ax = plt.subplots(figsize=(15, 6))
sns.histplot(labelstat_df, x='Classes', hue='NucleicAcid', weights='Voxel Count',
             multiple='dodge', shrink=0.8, palette = 'rainbow')

ax = plt.gca()
ax.set_yscale("log")
plt.show()

  0%|          | 0/5939 [00:00<?, ?it/s]

{'F': 0, 'fpocket_': 1, 'A': 2, 'C': 3, 'D': 4, 'G': 5, 'P': 6, 'R': 7, 'T': 8, 'U': 9, 'nucsite_': 10, 'nonsite_': 11, 'notnonsite': 12}


100%|██████████| 5939/5939 [45:06<00:00,  2.19it/s]


## Epilogue 

We have collected both the halo and their corresponding classes. In the next notebook, we will featurise the voxels. But before that, let's visualise some of them on voxels. I picked a few examples here.
* 5gmf - This is an example with internal symmetry in both Nucleotides and Proteins. I will visualise non-redundant sites and non-sites. We should expect only one of the "copies" to be highlighted in the voxel. Non-redundant site happens around z=36 (red), non-redundant non-site appears around z=72-144 (green)
* 3k62 - Fem-3-binding-factor 2. This is a pdb entry with no symmetry binding only one RNA chain. We should expect site and non-site to cover the whole surface of the pdb entry except a thin layer bordering the site and non-site around z=40
* 4f3t - Human Argonaute 2. This is a pdb entry with no symmetry that bind multiple chains of RNA. We should expect voronoi pattern in site classes.
* 2ez6 - Aquifex aeolicus Ribonuclease III. This is a pdb entry w/ a double stranded RNA, where majority of the base is enveloped by the backbone protected from protein contact. We should expect the halo to express this behaviour.


The cell below will save halo coordinate in xyz format in `junkfolder`, so that we can use pymol to inspect the quality of typification. 
> `pymol ../Database-PDB/cleansed/4f3t00000000.pdb {junkfolder}/A.xyz {junkfolder}/U.xyz {junkfolder}/C.xyz {junkfolder}/G.xyz`

Below `4f3t` is chosen. Can you catch which base is not interacting with the protein?


In [2]:

# ========================
# User input
# =======================
pdbid = '4f3t'
junkfolder = '/home/homingla/Downloads'



# =================== Imports

import sys
import glob
import tqdm
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append('../')
from scipy import sparse
from NucleicNet.DatasetBuilding.commandHalo import *
from NucleicNet.DatasetBuilding.util import WriteXyz




HaloC = Halo(HaloLowerBound = 2.5, HaloUpperBound = 5.0, LatticeSpacing = 1.0, 
             #n_HaloNearestNeighbor = 32, # NOTE Because ckdtree is fast I decide not to store it!
             DIR_OutputHaloTupleFolder = "../Database-PDB/halo",
             DIR_InputPdbFolder = "../Database-PDB/apo", 
             DIR_OutputTypifyFolder = "../Database-PDB/typo")

with open("../Database-PDB/typi/ClassIndex.pkl", "rb") as fn:
    tcmapping = pickle.load(fn)

# 5gmf

labelinterested = ['nonsite_', 'nucsite_']
protein_ = "../Database-PDB/apo/%s00000000.pdb" %(pdbid)
ppdb = PandasPdb()
CurrentPdbStructure = ppdb.read_pdb(protein_)
protein_df = CurrentPdbStructure.df['ATOM']
protein_coord =  protein_df[["x_coord", "y_coord", "z_coord"]].values
with open("../Database-PDB/halo/%s00000000.halotup"%(pdbid), "rb") as fn:
    halo_tuple = pickle.load(fn)
with np.load("../Database-PDB/typi/%s00000000.typi.npz" %(pdbid)) as f:
        spcm = sparse.csr_matrix((f['data'], f['indices'], f['indptr']), shape= f['shape'])
#print([tcmapping[i] for i in labelinterested])
print(spcm.sum(axis = 0).tolist())
loa = HaloC.RetrieveHaloCoords(  halo_tuple, reshape = True)
loa_ = loa[np.where(spcm[:,10].toarray() > 0)[0]]

WriteXyz(loa_,'N',"%s/nucs.xyz" %(junkfolder))
loa_ = loa[np.where(spcm[:,2].toarray() > 0)[0]]

WriteXyz(loa_,'A',"%s/A.xyz")
loa_ = loa[np.where(spcm[:,9].toarray() > 0)[0]]
WriteXyz(loa_,'U',"%s/U.xyz")
loa_ = loa[np.where(spcm[:,3].toarray() > 0)[0]]
WriteXyz(loa_,'C',"%s/C.xyz")
loa_ = loa[np.where(spcm[:,5].toarray() > 0)[0]]
WriteXyz(loa_,'G',"%s/G.xyz")
print(tcmapping)


#print(spcm_nuc)
spcm = spcm[:,[tcmapping[i] for i in labelinterested]]





# Direct visualisation 
HaloC.VisualiseHaloValues(halo_tuple, 
                    halo_values = spcm, protein_coord = protein_coord, Lattice_Bleeding = 5.0,
                    value_palette = "Jet", protein_palette = "Gray_r", 
                    halo_values_is_binaryclassmatrix = True)


#loa = HaloC.RetrieveHaloCoords(  halo_tuple, reshape = True)

#WriteXyz(loa,'N',"/home/homingla/Downloads/guagua.xyz")


[[11473.0, 11473.0, 283.0, 64.0, 0.0, 432.0, 1393.0, 1363.0, 0.0, 676.0, 4211.0, 83666.0, 5907.0]]
{'F': 0, 'fpocket_': 1, 'A': 2, 'C': 3, 'D': 4, 'G': 5, 'P': 6, 'R': 7, 'T': 8, 'U': 9, 'nucsite_': 10, 'nonsite_': 11, 'notnonsite': 12}


# Epilogue

In the first implementation of NucleicNet, all calculations were done on voxels from a regular cubic lattice and, as a pilot study, we did not implement data structure that allows communication among voxels, because that will require data augmentation to remedy the lifting of rotational/permutation invariance. Here, we do something more by giving the indices of the halo; this allow fast search of neighboring halos on a kDTree. Some directions:
* Coordinates. Rather than storing as indices (integers), we also stored coordinate (floats) of halo as `.haloxyz`. Both the indices and coordinates will work on a kDTree to catch neighboring voxels. The plus here for storing the coordinate is that it will facillitate development of collecting features on mesh (rather than regular lattice). The mesh can be an isosurface triangulation or some other level set. But note that a simple [Marching Cube](https://en.wikipedia.org/wiki/Marching_cubes) isosurface is not necessarily translational invariant.
* Pocket Landmarks. Currently, the time profile bottleneck is on the use of Fpocket. Maybe we should implement a light-weight [alpha sphere algorithm](https://graphics.stanford.edu/courses/cs268-11-spring/handouts/AlphaShapes/as_fisher.pdf) to simplify the process. 
* Bounds of Halo. While a lower bound at 2.5 angstrom make perfect sense in ML models with no message passing among voxels as reasoned above, when the restriction on message passing is lifted, those voxels between 1.25 and 2.5 angstrom with lower sparsity in the feature vector may provide more textured information to those voxels further away.  